In [12]:
# !pip install gymnasium renderlab
# !pip install opencv-python
# !pip install pygame

In [13]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [14]:
#visualise maze:
rfpMaze = ["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]
maze1 = ["SFFH", "FHHF", "FHFG", "FFFH", "HFHH"]

desc = rfpMaze
mazeSize = [len(desc),len(desc[0])]

statePositions = [[] for _ in range(mazeSize[0])]
stateNum = 0
for i in range(mazeSize[0]):
    for j in range(mazeSize[1]):
        statePositions[i].append(stateNum)
        stateNum += 1
        

        
giftState = -1
gift_found = False
for i in range(len(desc)):
    if gift_found:
        break
    for j in range(len(desc[i])):
        giftState += 1
        if desc[i][j] == 'G':
            gift_found = True
            break
            
print(giftState)
print(statePositions)

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human") 

18
[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15], [16, 17, 18, 19]]


In [40]:
def resetTable():
    global qTable_1
    qTable_1 = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        qTable_1[i] = [0,0,0,0] 
    global currentState
    currentState = 0
    
def getPosition(state):
    for i in range(len(statePositions)):
        for j in range(len(statePositions[i])):
            if statePositions[i][j] == state:
                return i, j
            
def calcReward(state, nextState):
    y1, x1 = getPosition(state)
    y2, x2 = getPosition(nextState)
    y3, x3 = getPosition(giftState)
    
    currentDist = (((y3 - y1)**2)+((x3 - x1)**2))**0.5
    nextDist = (((y3 - y2)**2)+((x3 - x2)**2))**0.5
    
    changeInDist = currentDist-nextDist
    if changeInDist > 0:
        return changeInDist/2
    else:
        return changeInDist

def calcPossibleMoves(state):
    global qTable_1
    possibleMoves = []
    
    if state == 0:
        return [1,2]
    
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(2)
        
    if (state+1) % mazeSize[1] != 1:
        possibleMoves.append(0)
        
    if state > (mazeSize[1]-1):
        possibleMoves.append(3)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(1)
        
    return possibleMoves

def nextStep(state):
    global qTable_1
    possMoves = calcPossibleMoves(state)
    
    if random.random() < epsilonValue:
        nextMove = random.choice(possMoves)
    else:
        qValues = {}
        for move in possMoves:
            qValues[move] = qTable_1[state][move]

        maxValue = max(qValues.values())
        minValue = min(qValues.values())
        count_max = sum(1 for value in qValues.values() if value == maxValue)
        count_min = sum(1 for value in qValues.values() if value == minValue)

        if count_max > 1 and count_max < len(possMoves):
            nextMove = random.choice([move for move in possMoves if qValues[move] != minValue])
        elif count_max == len(possMoves):
            nextMove = random.choice(possMoves)
        else:
            nextMove = max(qValues, key=qValues.get)
    return nextMove

def pathFound():
    currentState = 0
    for i in range(mazeSize[0]*mazeSize[1]-1):
        bestDirection = None
        if max(qTable_1[currentState]) > 0:
            bestDirection = qTable_1[currentState].index(max(qTable_1[currentState]))
        newState = 0
        if bestDirection == 0 and 0 in calcPossibleMoves(currentState):
            newState = currentState - 1
        elif bestDirection == 1 and 1 in calcPossibleMoves(currentState):
            newState = currentState + mazeSize[1]
        elif bestDirection == 2 and 2 in calcPossibleMoves(currentState):
            newState = currentState + 1
        elif bestDirection == 3 and 3 in calcPossibleMoves(currentState):
            newState = currentState - mazeSize[1]

        if newState == giftState:
            return True
        currentState = newState
    return False

convergenceThresh = 0.001
def updateTable_qLearning(direction, nextState, reward):
    global qTable_1
    global currentState
    didConverge = False
    updated = qTable_1[currentState][direction] + alpha*(reward + (gamma*max(qTable_1[nextState])) - qTable_1[currentState][direction])
    changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
    if changeInQ < convergenceThresh:
        if changeInQ > 0 and pathFound():
            didConverge = True
    qTable_1[currentState][direction] = updated
    timesVisited[currentState] += 1
    currentState = nextState
    return didConverge, changeInQ

def updateTable_sarsa(direction, nextState, reward):
    global qTable_1
    global currentState
    didConverge = False
    nextDirection = nextStep(nextState)
    updated = qTable_1[currentState][direction] + alpha*(reward + (gamma*qTable_1[nextState][nextDirection]) - qTable_1[currentState][direction])
    changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
    if changeInQ < convergenceThresh:
        if changeInQ > 0 and pathFound():
            didConverge = True
    qTable_1[currentState][direction] = updated
    timesVisited[currentState] += 1
    currentState = nextState
    return didConverge, changeInQ, nextDirection

In [43]:
#Q LEARNING-----------------------------------------------------------------------

epsilonValue = 0.2
alpha = 0.6
gamma = 0.7
qTable_1 = {}
currentState = 0

maxEpisodes = 1000
currentEpisode = 1
converged = False
revistPenalty = -0.25

timesVisited = {}
def resetVisited():
    global timesVisited
    timesVisited = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        timesVisited[i] = 0
resetVisited()

resetTable()
env.reset()
start_time = time.time()
while currentEpisode <= maxEpisodes:
    global currentSteps
    if converged:
        break
        
    direction = nextStep(currentState)
    nextState, reward, terminated, truncated, info = env.step(direction)
    
    if terminated:
        if reward < 1:
            reward = -10
        else:
            reward = 10
            
    if not terminated:
        reward = calcReward(currentState, nextState)
        if timesVisited[nextState] > 0:
            reward += revistPenalty*timesVisited[nextState]
    
    converged, changeInQ = updateTable_qLearning(direction, nextState, reward)
    

    if terminated or truncated or converged:
        observation, info = env.reset()
        if not converged:
            currentEpisode += 1
            resetVisited()

        
    if converged:
        end_time = time.time()

    clear_output(wait=True)
    print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
    print("Time: " + str(round(time.time()-start_time, 3)) + " sec")
    print("Q-Table:")
    for i in range(len(qTable_1)):
        print(str(i) + ": " + str(qTable_1[i]))
    print("change in Q: " + str(changeInQ))
            
if converged:
    duration = end_time - start_time
    print(str(round(duration, 3)) + " seconds to converge")
else:
    print("No convergence")

Episode: 24/1000
Time: 45.865 sec
Q-Table:
0: [0, 2.3638337905590743, -0.04945898275534358, 0]
1: [0.1680593114299517, -8.4, -0.45238656721093484, 0]
2: [-0.4572042019877048, -6.0, -0.07386337537059635, 0]
3: [-0.2630683123147018, 0, 0, 0]
4: [0, 3.112542045320885, -9.060333038715946, 0.513109238250589]
5: [-0.4891894695428616, 1.5342124867364557, -0.5764967792695687, -0.7419371553965983]
6: [-0.8832815729997477, -0.15518515860183427, 0, 0]
7: [0, 0, 0, 0]
8: [0, 0, 4.035041549060179, -0.3675167401970312]
9: [-0.3538612467110829, 5.427584760984712, 0.11047981346990163, -8.4]
10: [-0.27346964865898915, -6.0, -0.14164078649987388, -6.0]
11: [-0.07917960675006305, 0, 0, 0]
12: [0, -0.6532576325196203, 0, 0]
13: [-6.0, 7.201933826831344, -6.0, 0.7843502328516583]
14: [-1.2987555355451872, 0, -1.8738633753705962, 0]
15: [0, 0, 0, 0]
16: [0, 0, 0.15, 0]
17: [-0.6, 0, 9.999580569599999, -0.3463372285648471]
18: [-2.9547515034648644, 0, -3.4106439598260345, -3.4336600645270314]
19: [0, 0, 0, 0

In [44]:
#SARSA-----------------------------------------------------------------------

epsilonValue = 0.35
alpha = 0.6
gamma = 0.7
qTable_1 = {}
currentState = 0

maxEpisodes = 1000
currentEpisode = 1
converged = False
revistPenalty = -0.25
startOfEpisode = True
direction = None

timesVisited = {}
def resetVisited():
    global timesVisited
    timesVisited = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        timesVisited[i] = 0
resetVisited()

resetTable()
env.reset()
start_time = time.time()
while currentEpisode <= maxEpisodes:
    global startOfEpisode
    global currentSteps
    global Direction
    if converged:
        break
        
    if startOfEpisode:
        direction = nextStep(currentState)
        startOfEpisode = False
    nextState, reward, terminated, truncated, info = env.step(direction)
    
    if terminated:
        if reward < 1:
            reward = -10
        else:
            reward = 10
            
    if not terminated:
        reward = calcReward(currentState, nextState)
        if timesVisited[nextState] > 0:
            reward += revistPenalty*timesVisited[nextState]
    
    converged, changeInQ, nextDirection = updateTable_sarsa(direction, nextState, reward)
    direction = nextDirection

    if terminated or truncated or converged:
        observation, info = env.reset()
        if not converged:
            currentEpisode += 1
            startOfEpisode = False
            resetVisited()

        
    if converged:
        end_time = time.time()

    clear_output(wait=True)
    print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
    print("Time: " + str(round(time.time()-start_time, 3)) + " sec")
    print("Q-Table:")
    for i in range(len(qTable_1)):
        print(str(i) + ": " + str(qTable_1[i]))
    print("change in Q: " + str(changeInQ))
            
if converged:
    duration = end_time - start_time
    print(str(round(duration, 3)) + " seconds to converge")
else:
    print("No convergence")

Episode: 37/1000
Time: 84.435 sec
Q-Table:
0: [0, -6.51813646359424, -2.3563427005296944, 0]
1: [-2.3991116509521078, -9.356901792447362, -2.2961538310610043, 0]
2: [-2.911993969342743, -9.780620641270348, -2.9741851899127605, 0]
3: [-6.322774061965752, -8.4, 0, 0]
4: [0, -5.001383268205869, -11.051672584939716, -2.297761115656176]
5: [-1.5324375161601012, -4.379316553908443, -1.6940952780820075, -3.780831161461309]
6: [-0.8665907239651015, -1.5769089277109225, -0.6738633753705964, -1.8542049653578594]
7: [-1.456941605237573, -3.8980608183341166, 0, -0.5364777760781874]
8: [0, -9.82597963509649, -3.3648180755996315, -5.2263075390137566]
9: [-0.8338871469097379, -0.12647539805523744, -1.1006165533707695, -9.4588578895895]
10: [-2.090357640309518, -6.0, -3.6671395425169755, -6.0]
11: [0, 0.4525983799087356, 0, -9.608681723533381]
12: [0, -2.093220285281596, -2.177329456221495, -1.2517752490297958]
13: [-10.198478361927588, -1.455289870527411, -6.0, -3.5129172175143024]
14: [0, 0, -1.8869

KeyboardInterrupt: 

In [6]:
env.reset()

(0, {'prob': 1})

In [7]:
env.close()